In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from multiprocessing import Pool
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [2]:
# Load data
data_path = "../../data"
X_train = pd.read_csv(os.path.join(data_path, "X_train.csv"))
y_train_org = pd.read_csv(os.path.join(data_path, "y_train.csv"))
X_test = pd.read_csv(os.path.join(data_path, "X_test.csv"))
y_test_org = pd.read_csv(os.path.join(data_path, "y_test.csv"))

In [3]:
traits = ['Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional Stability', 'Openness']
random_state=27

In [4]:
# Create results directory
results_path = "../../results"
specific_results_path = os.path.join("../../results", "knn_classification_non_pca")
os.makedirs(results_path, exist_ok=True)
os.makedirs(specific_results_path, exist_ok=True)

In [5]:
def calc_roc_auc(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    n_classes = np.unique(y_true)
    roc_auc_scores = []
    for label in n_classes:
        # Create binary labels for the current class vs. all other classes
        y_true_class = (y_true == label).astype(int)
        y_pred_class = (y_pred == label).astype(int)
        
        # Calculate ROC AUC for the current class
        roc_auc = roc_auc_score(y_true_class, y_pred_class)
        roc_auc_scores.append(roc_auc)
    return roc_auc_scores

In [6]:
# Define the parameter grid you want to search over
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],      # Weighting scheme for neighbors
    'p': [1, 2],                             # Distance metric (1 for Manhattan distance, 2 for Euclidean distance)
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute nearest neighbors
    'leaf_size': [10, 20, 30, 40, 50],       # Leaf size for tree-based algorithms
    'metric': ['minkowski', 'manhattan', 'euclidean', 'chebyshev']  # Distance metric for Minkowski distance
}

# Create a KNN classifier
knn = KNeighborsClassifier()

# Create a grid search object
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, scoring="accuracy", cv=3, n_jobs=5)

for trait in traits:
    print(f"Processing {trait}")
    trait_bin = trait + "_bin"
    label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
    y_train = [label_mapping[label] for label in y_train_org[trait_bin]]
    y_test = [label_mapping[label] for label in y_test_org[trait_bin]]
    # Fit the grid search to your data
    grid_search.fit(X_train, y_train)

    # Print the best hyperparameters and the corresponding score
    print("Best Hyperparameters: ", grid_search.best_params_)
    print("Best Score: ", grid_search.best_score_)

    # Get the best model from the grid search
    best_knn = grid_search.best_estimator_

    # Now, you can use the best_rf model for predictions on your test data
    y_pred = best_knn.predict(X_test)
    y_true = y_test

    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")
    roc_auc = calc_roc_auc(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')
    print(f'ROC AUC: {roc_auc}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    print("\n\n")
    metrics = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1, "roc_auc": roc_auc, "conf_matrix": conf_matrix, "best_hyperparameters": grid_search.best_params_, "best_score": grid_search.best_score_}

    # Save model and metrics 
    curr_result_path = os.path.join(specific_results_path, trait)
    os.makedirs(curr_result_path, exist_ok=True)
    with open(os.path.join(curr_result_path, f'knn_model_tuned.pkl'), 'wb') as file:
        pickle.dump(best_knn, file)
    with open(os.path.join(curr_result_path, f'perf_metrics_tuned.pkl'), 'wb') as file:
        pickle.dump(metrics, file)

Processing Extraversion
Best Hyperparameters:  {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
Best Score:  0.6415195291599786
Accuracy: 0.6346153846153846
Precision: 0.6291721927497789
Recall: 0.6346153846153846
F1-Score: 0.6269467972334378
ROC AUC: [0.650887573964497, 0.6962894248608534, 0.7191323121209006]
Confusion Matrix:
[[15 18  6]
 [ 8 75 15]
 [ 6 23 42]]



Processing Agreeableness
Best Hyperparameters:  {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
Best Score:  0.7191011235955056
Accuracy: 0.7403846153846154
Precision: 0.737777573303889
Recall: 0.7403846153846154
F1-Score: 0.7379507211538462
ROC AUC: [0.7227342549923195, 0.7227342549923195]
Confusion Matrix:
[[ 53  31]
 [ 23 101]]



Processing Conscientiousness
Best Hyperparameters:  {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
Best S